# Self-Reflective RAG Agent

![Self-Reflective RAG](https://blog.langchain.com/content/images/size/w1000/2024/02/image-6.png)

- [LangChain 문서](https://blog.langchain.com/agentic-rag-with-langgraph/)
- LLM 의 답변을 LLM 이 다시 검증하는 방식
- 여기서는 `ollama` 를 사용함
    - embedding model: `bge-m3:567m`
    - llm model: `gpt-oss:120b-cloud`

## 0. Helper 함수

- jupyter notebook 에서 graph 보여주는 함수

In [1]:
import uuid

from IPython.display import HTML
from langchain_core.runnables.graph import Graph


def draw_mermaid_with_html(g: Graph):
    mermaid_code = g.draw_mermaid()

    container_id = f"mermaid-container-{uuid.uuid4().hex[:8]}"

    html = f"""
    <div id="{container_id}"></div>

    <script type="module">
        import mermaid from 'https://cdn.jsdelivr.net/npm/mermaid@10/dist/mermaid.esm.min.mjs';

        mermaid.initialize({{
            startOnLoad: false,
            theme: 'default',
            securityLevel: 'loose'
        }});

        const graphDefinition = `{mermaid_code}`;
        const container = document.getElementById('{container_id}');

        try {{
            const {{ svg }} = await mermaid.render('graphDiv-' + Date.now(), graphDefinition);
            container.innerHTML = svg;
        }} catch (error) {{
            container.innerHTML = '<pre>' + error + '</pre>';
        }}
    </script>
    """

    display(HTML(html))

## 1. llm 및 retriever 정의

In [2]:
from pathlib import Path

from langchain_chroma import Chroma
from langchain_ollama import ChatOllama, OllamaEmbeddings


DATA_DIR = Path("../data")

MODEL_EMBEDDINGS = "bge-m3:567m"
MODEL_LLM = "gpt-oss:120b-cloud"

VECTOR_DB_PERSIST_DIR = DATA_DIR / "chroma_db"
VECTOR_DB_COLLECTION_NAME = "korean_income_tax_law"

embedding_function = OllamaEmbeddings(model=MODEL_EMBEDDINGS)

vector_store = Chroma(
    persist_directory=VECTOR_DB_PERSIST_DIR,
    embedding_function=embedding_function,
    collection_name=VECTOR_DB_COLLECTION_NAME,
)

income_tax_law_retriever = vector_store.as_retriever(kwargs={"k": 1})  # 검색 결과는 1개만

llm = ChatOllama(model=MODEL_LLM)

## 2. State 및 Builder 만들기

In [3]:
from typing import TypedDict

from langchain_core.documents import Document
from langgraph.graph import StateGraph


class AgentState(TypedDict):
    query: str
    context: list[Document]
    answer: str


graph_builder = StateGraph(AgentState)

## 3. Node 용 함수 만들기

In [4]:
from typing import Literal

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

In [5]:
def retrieve(state: AgentState) -> AgentState:
    query = state["query"]
    retrieved_docs = income_tax_law_retriever.invoke(query)
    return {"context": retrieved_docs}


retrieve_result = retrieve({"query": "연봉 5천만원인 거주자의 소득세는 얼마야?"})
retrieve_result

{'context': [Document(id='cdbd2f99-818e-4bf0-a5b7-a31e94687d99', metadata={'start_index': 85516, 'source': '../data/income-tax-law-parsed-by-upstage.md'}, page_content='제1항에 따른 해당 거주자의 보유기간의 이자등 상당액에 대한 세액으로 한정한다)5. 제150조에 따른 납세조합의 징수세액과 그 공제액'),
  Document(id='22ed77a4-2054-46d1-a6d6-fcc2a995584f', metadata={'source': '../data/income-tax-law-parsed-by-upstage.md', 'start_index': 21510}, page_content='과세기간의 총수입금액에서 이에 사용된 필요경비를 공제한 금액으로 하며, 필요경비가 총 수입금액을 초과하는 경우 그 초과하는 금액을 "결손금"이라 한다. ③ 제1항 각 호에 따른 사업의 범위에 관하여는 이 법에 특별한 규정이 있는 경우 외에는 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준산업분류에 따르고, 그 밖의 사업소득의 범위에 관하여 필요한 사항은 대통령령으로 정한다.# [전문개정 2009. 12. 31.]제20조(근로소득) ① 근로소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 한다. <개정 2016. 12. 20., 2024. 12. 31.>- 1. 근로를 제공함으로써 받는 봉급 · 급료 · 보수 · 세비 · 임금 · 상여 · 수당과 이와 유사한 성질의 급여 - 2. 법인의 주주총회 · 사원총회 또는 이에 준하는 의결기관의 결의에 따라 상여로 받는 소득 - 3. 「법인세법」에 따라 상여로 처분된 금액 - 4. 퇴직함으로써 받는 소득으로서 퇴직소득에 속하지 아니하는 소득 - 5. 종업원등 또는 대학의 교직원이 지급받는 직무발명보상금(제21조제1항제22호의2에 따른 직무발명보상금은 제 - 외한다) - 6. 사업자나 법인이 생산 · 

In [6]:
def generate(state: AgentState) -> AgentState:
    query = state["query"]
    raw_context = state["context"]

    prompt = PromptTemplate.from_template("""역할: 당신은 한국의 소득세법 전문가입니다.
---
지시: 오직 주어진 정보를 기반으로 사용자의 질문에 답하세요.
---
정보: {context}
---
사용자 질문: {query}""")
    context = "\n\n".join([doc.page_content for doc in raw_context])
    rag_chain = prompt | llm | StrOutputParser()
    answer = rag_chain.invoke({"query": query, "context": context})
    return {"answer": answer}


generate_result = generate({"query": "연봉 5천만원인 거주자의 소득세는 얼마야?", "context": retrieve_result["context"]})
generate_result

{'answer': '안녕하세요. 문의해 주신 “연봉\u202f5천만원인 거주자의 소득세는 얼마인가?”에 대해 답변드리겠습니다.\n\n### 1. 소득세 계산 흐름 (제20조·제47조 기준)\n\n1. **총급여액(연봉) → 근로소득금액**  \n   - 연봉\u202f5천만원은 “총급여액”(비과세소득을 제외한 금액)으로 간주됩니다.  \n   - 여기서 **근로소득공제(제47조)** 를 차감하면 최종적인 **근로소득금액**이 산출됩니다.\n\n2. **과세표준 산출**  \n   - 근로소득금액에서 **인적공제·특별공제·근로소득세액공제** 등(법령에 따름) 를 추가로 차감하면 **과세표준**이 됩니다.\n\n3. **세율 적용**  \n   - 과세표준에 **소득세율(구간별 누진세율)** 을 적용하여 **산출세액**을 구합니다.  \n   - 산출세액에서 이미 원천징수된 세액(급여에서 원천 징수된 세액)을 차감하면 **납부할 세액**이 확정됩니다.\n\n### 2. 현재 제공된 정보에 없는 내용\n\n- **근로소득공제액**(제47조에 정해진 구체적 계산식)  \n- **인적·특별공제** 등 세액을 감소시키는 항목들의 구체적인 금액  \n- **소득세율표(구간별 세율 및 누진공제액)**  \n\n위 항목들은 **소득세법·소득세법 시행령·국세청 고시** 등에 따로 규정되어 있으며, 현재 대화에 포함되지 않았습니다.\n\n### 3. 결론\n\n제공된 조문만으로는  \n\n- **근로소득공제액**  \n- **세율(구간별 누진세율 및 누진공제액)**  \n\n등을 알 수 없으므로, 정확한 **소득세 금액**을 산출할 수 없습니다.  \n\n다만, 일반적인 계산 절차는 아래와 같습니다.\n\n| 단계 | 내용 |\n|------|------|\n| 1 | **총급여액**\u202f=\u202f50,000,000\u202f원 |\n| 2 | **근로소득공제**\u202f=\u202f(법정 계산식에 따라 차감) |\n| 3 | **근로소득금액**\u202f=\u202

In [7]:
llm_for_checking = ChatOllama(model=MODEL_LLM, temperature=0)  # 최대한 같은 답이 나올 수 있도록

In [8]:
def check_doc_relevance(state: AgentState) -> Literal["irrelevant", "relevant"]:
    query = state["query"]
    context = state["context"]
    prompt = PromptTemplate.from_template("""역할: '검색된 문서 내용'과 '사용자의 질문'이 얼마나 연관이 있는지 등급 매기기입니다.
---
지시: 문서가 사용자 질문과 키워드나 의미적으로 관련이 있으면 1, 없으면 0을 반환하세요. 엄격한 기준이 아닌, 명백히 무관한 문서만 필터링하는 것이 목적입니다.
---
출력: 관련이 있으면 'relevant', 없으면 'irrelevant'
---
검색된 문서 내용:
{documents}
---
사용자 질문: {question}""")
    doc_relevance_chain = prompt | llm_for_checking | StrOutputParser()
    return doc_relevance_chain.invoke(
        {
            "question": query,
            "documents": context,
        }
    )


relevance_result = check_doc_relevance(
    {
        "query": "연봉 5천만원인 직장인의 소득세는 얼마야?",
        "context": retrieve_result["context"],
    }
)
relevance_result

'relevant'

In [9]:
def rewrite(state: AgentState) -> AgentState:
    original_query = state["query"]
    dictionary = ["- 사람을 나타내는 표현 -> 거주자"]
    prompt = PromptTemplate.from_template(f"""역할: 당신은 한국의 소득세법 VectorDB 질의로 질문을 변환하는 변환기입니다.
---
지시: 사용자의 질문을 보고, 우리의 사전을 참고해서 소득세법이 들어있는 VectorDB 에서 Document 가 잘 검색될 수 있도록 사용자의 질문을 변경해주세요.
---
사전:
{dictionary}
---
출력: 변경된 질문 내용
---
질문: {{query}}
""")
    rewrite_query_chain = prompt | llm | StrOutputParser()
    return {"query": rewrite_query_chain.invoke(original_query)}


rewrite_result = rewrite({"query": "연봉 5천만원인 사람의 소득세는?"})
rewrite_result

{'query': '연봉 5천만원인 거주자의 소득세는?'}

In [10]:
def check_hallucination(state: AgentState) -> Literal["not-hallucinated", "hallucinated"]:
    answer = state["answer"]
    raw_context = state["context"]
    prompt = PromptTemplate.from_template("""역할: '학생의 답변'이 오직 '문서 내용'에 기반한 답변인지 아닌지를 판별하는 선생님입니다.
---
출력: 오직 문서 내용에 대해서만 근거한다면 'not-hallucinated', 아니면 'hallucinated'
---
문서 내용: {documents}
---
학생의 답변: {student_answer}""")
    check_chain = prompt | llm_for_checking | StrOutputParser()

    context = "\n\n".join([doc.page_content for doc in raw_context])
    return check_chain.invoke({"documents": context, "student_answer": answer})


hallucination_result = check_hallucination({"answer": generate_result["answer"], "context": retrieve_result["context"]})
hallucination_result

'hallucinated'

In [11]:
def check_useful_answer(state: AgentState) -> Literal["unuseful", "useful"]:
    query = state["query"]
    answer = state["answer"]
    prompt = PromptTemplate.from_template("""지시: '답변'이 '질문'에 대해서 유용한지를 판단하세요.
---
출력: 유용한 답변이면 'useful', 아니면 'unuseful'
---
질문: {query}
---
답변: {answer}""")
    check_chain = prompt | llm_for_checking | StrOutputParser()
    return check_chain.invoke({"query": query, "answer": answer})


useful_answer_result = check_useful_answer(
    {
        "query": rewrite_result["query"],
        "answer": generate_result["answer"],
    }
)
useful_answer_result

'unuseful'

## 3. Edge 추가하기

![Self-Reflective RAG](https://blog.langchain.com/content/images/size/w1000/2024/02/image-6.png)


In [12]:
graph_builder.add_node("retrieve", retrieve)
graph_builder.add_node("rewrite", rewrite)
graph_builder.add_node("generate", generate)


def passthrough(_: AgentState) -> AgentState:  # 조건 edge 를 바로 연결할 수는 없음
    return {}  # 상태 변경 없음


graph_builder.add_node("passthrough", passthrough)

In [13]:
from langgraph.graph import END, START


graph_builder.add_edge(START, "retrieve")
graph_builder.add_conditional_edges(
    "retrieve",
    check_doc_relevance,
    {
        "irrelevant": END,
        "relevant": "generate",
    },
)
graph_builder.add_conditional_edges(
    "generate",
    check_hallucination,
    {
        "not-hallucinated": "passthrough",
        "hallucinated": END,
    },
)
graph_builder.add_conditional_edges(
    "passthrough",
    check_useful_answer,
    {
        "unuseful": "rewrite",
        "useful": END,
    },
)

graph_builder.add_edge("rewrite", "retrieve")

In [14]:
self_rag_graph = graph_builder.compile()
draw_mermaid_with_html(self_rag_graph.get_graph())

In [15]:
self_rag_graph.invoke({"query": "연봉 5천만원인 직장인의 소득세는?"})

{'query': '연봉 5천만원인 직장인의 소득세는?',
 'context': [Document(id='22ed77a4-2054-46d1-a6d6-fcc2a995584f', metadata={'source': '../data/income-tax-law-parsed-by-upstage.md', 'start_index': 21510}, page_content='과세기간의 총수입금액에서 이에 사용된 필요경비를 공제한 금액으로 하며, 필요경비가 총 수입금액을 초과하는 경우 그 초과하는 금액을 "결손금"이라 한다. ③ 제1항 각 호에 따른 사업의 범위에 관하여는 이 법에 특별한 규정이 있는 경우 외에는 「통계법」 제22조에 따라 통계청장이 고시하는 한국표준산업분류에 따르고, 그 밖의 사업소득의 범위에 관하여 필요한 사항은 대통령령으로 정한다.# [전문개정 2009. 12. 31.]제20조(근로소득) ① 근로소득은 해당 과세기간에 발생한 다음 각 호의 소득으로 한다. <개정 2016. 12. 20., 2024. 12. 31.>- 1. 근로를 제공함으로써 받는 봉급 · 급료 · 보수 · 세비 · 임금 · 상여 · 수당과 이와 유사한 성질의 급여 - 2. 법인의 주주총회 · 사원총회 또는 이에 준하는 의결기관의 결의에 따라 상여로 받는 소득 - 3. 「법인세법」에 따라 상여로 처분된 금액 - 4. 퇴직함으로써 받는 소득으로서 퇴직소득에 속하지 아니하는 소득 - 5. 종업원등 또는 대학의 교직원이 지급받는 직무발명보상금(제21조제1항제22호의2에 따른 직무발명보상금은 제 - 외한다) - 6. 사업자나 법인이 생산 · 공급하는 재화 또는 용역을 그 사업자나 법인(「독점규제 및 공정거래에 관한 법률」 에 따 - 른 계열회사를 포함한다)의 사업장에 종사하는 임원등에게 대통령령으로 정하는 바에 따라 시가보다 낮은 가격 - 으로 제공하거나 구입할 수 있도록 지원함으로써 해당 임원등이 얻는 이익 - ② 근로소득금액은 제1항 각 호의 소득의 금액의 합계액(비과세소득의 금액은 제외하며, 